# Mixtures of conjugate priors 01

# Stan treatment

# _Josep Fortiana_ $\hskip6cm$ 2022-04-04

## The spinning coin

***
###### LaTeX macros
$\def\prob{P}$
$\def\argmax{\operatorname{arg\,max}}$
$\def\argmin{\operatorname{arg\,min}}$
$\def\borel{\operatorname{Borel}}$
$\def\cE{\cal E}$
$\def\cP{\cal P}$
$\def\R{\mathbb{R}}$ 
$\def\N{\mathbb{N}}$
$\def\Z{\mathbb{Z}}$
$\def\Ee{\operatorname{E}}$
$\def\va{\text{v.a.}}$
$\def\var{\operatorname{var}}$
$\def\cov{\operatorname{cov}}$
$\def\cor{\operatorname{cor}}$
$\def\binomdist{\operatorname{Binom}}$
$\def\berndist{\operatorname{Ber}}$
$\def\betabinomdist{\operatorname{Beta-Binom}}$
$\def\betadist{\operatorname{Beta}}$
$\def\expdist{\operatorname{Exp}}$
$\def\gammadist{\operatorname{Gamma}}$
$\def\hyperdist{\operatorname{Hypergeom}}$
$\def\hypergeomdist{\operatorname{Hypergeom}}$
$\DeclareMathOperator{\multinomialdist}{Multinomial}$
$\DeclareMathOperator{\multinomdist}{Multinom}$
$\def\poissondist{\operatorname{Poisson}}$
$\def\geomdist{\operatorname{Geom}}$
$\def\normaldist{\operatorname{N}}$
$\def\unifdist{\operatorname{Unif}}$
$\DeclareMathOperator{\indica}{\mathbb{1}}$
$\def\CondTo{\mathbin{|\mskip0.5mu}}$
***

In [ ]:
#install.packages("rstan", dependencies=TRUE,repos= "https://cloud.r-project.org")
require(rstan)

In [ ]:
# Following directions:
# For execution on a local, multicore CPU with excess RAM we recommend calling
# options(mc.cores = parallel::detectCores()).
# To avoid recompilation of unchanged Stan programs, we recommend calling
# rstan_options(auto_write = TRUE)
options(mc.cores = parallel::detectCores())
rstan_options(auto_write = TRUE)

## Specify model

Stan is unable to simulate discrete parameters.

One possible workaround is to generate on the spot the unnamed index selecting which of both components
in the mixture is chosen:

If $u\sim\unifdist(0,1)$ and $\gamma\in(0,1)$ then $\prob(u<\gamma)=\gamma$.

We implement this with the ternary infix conditional operator.

In [ ]:
modelString = "
    data{
        int<lower=0> n ;
        int<lower=0> x ; 
        real<lower=0> a1 ;
        real<lower=0> b1 ;
        real<lower=0> a2 ;
        real<lower=0> b2 ;
        real<lower=0,upper=1>  gamma ;
        }
    parameters{
        real<lower=0,upper=1>  u ;
        vector<lower=0,upper=1>[2] theta ;
        }
    transformed parameters{                         
        real<lower=0,upper=1> p ;                   
                                                    // Here the trick:
                                                    // Generate on the spot the unnamed index selecting 
                                                    // which of both components in the mixture is chosen
        p=theta[u<gamma?1:2] ;
        }
    model{
        x ~ binomial(n,p) ;
        theta[1]~beta(a1,b1) ;
        theta[2]~beta(a2,b2) ;
        u ~ uniform(0,1) ; 
        }"

## Compile model and sample from the posterior pdf

In [ ]:
# Translate model to C++ and compile to DSO:
stanDso <- stan_model( model_code=modelString ) 

## Data 

In [ ]:
prior.alpha1<-10
prior.beta1<-20
prior.alpha2<-20
prior.beta2<-10
prior.gamma<-0.5

In [ ]:
# Number of trials
n<-10
# Observed x
x.obs<-3

In [ ]:
Mix.01.Standat<-list(a1=prior.alpha1,b1=prior.beta1,a2=prior.alpha2,b2=prior.beta2,gamma=prior.gamma,n=n,x=x.obs)

In [ ]:
# Generate posterior sample:
stanFit <- sampling( object=stanDso, 
                     data = Mix.01.Standat, 
                     chains = 3,
                     iter = 4000, 
                     warmup = 200, 
                     thin = 1)

In [ ]:
S<-summary(stanFit)
round(S$summary,5)

# Diagnostic diagrams with the `bayesplot` package

In [ ]:
#install.packages("bayesplot", dependencies=TRUE,repos= "https://cloud.r-project.org")
require(bayesplot)

In [ ]:
color_scheme_set("green")
options(repr.plot.width=10,repr.plot.height=6)
mcmc_trace(stanFit, pars = c("p"))

# Analysis of posterior pdf properties

### Posterior credible interval 

In [ ]:
color_scheme_set("green")
options(repr.plot.width=7,repr.plot.height=3)
mcmc_intervals(stanFit, pars = c("p","theta[1]","theta[2]"),prob=0.80,prob_outer=0.95)
# Defaults are
# prob = 0.5,
# prob_outer = 0.9,

### Areas diagram

In [ ]:
color_scheme_set("green")
options(repr.plot.width=7,repr.plot.height=7)
mcmc_areas(stanFit, pars = c("p","theta[1]","theta[2]"),prob=0.80,prob_outer=0.95)

### Histogram

In [ ]:
color_scheme_set("green")
options(repr.plot.width=15,repr.plot.height=5)
mcmc_hist(stanFit, pars = c("p","theta[1]","theta[2]"),binwidth=0.05)

### Density plot

In [ ]:
options(repr.plot.width=15,repr.plot.height=5)
mcmc_dens(stanFit, pars = c("p","theta[1]","theta[2]"))

In [ ]:
color_scheme_set("green")
options(repr.plot.width=5,repr.plot.height=5)
mcmc_dens_chains(stanFit, pars = c("p","theta[1]","theta[2]"))

### Violin plot

In [ ]:
color_scheme_set("green")
options(repr.plot.width=15,repr.plot.height=5)
mcmc_violin(stanFit, pars = c("p","theta[1]","theta[2]"))